# Remoção das stopwords, limpeza do texto e tokenização

## Importações

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!python -m spacy download pt

     |████████████████████████████████| 21.2MB 1.2MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186282 sha256=1e8427fb441d23f1d3beb34c25bce8e950a6025c9dff9295900dcbe36c13771d
  Stored in directory: /tmp/pip-ephem-wheel-cache-pedjiib5/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [ ]:
import numpy as np
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

# import spacy
# nlp = spacy.load('pt')

## Leitura dos dados

In [ ]:
df = pd.read_csv('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/02. Arquivos/01. Dados/training_set_fakenews.csv', header=0, names=['text', 'classification'], index_col=0)
# df = df[['text', 'y']]
df

,text,classification
0,China se beneficiou financeiramente do novo co...,Falso
1,Alimentos alcalinos ajudam a prevenir ou trata...,Falso
2,"Minas Gerais tem menos mortes que São Paulo, e...",Enganoso
3,"Quando os sintomas do COVID-19 aparecerem, as ...",Falso
4,Beber água e fazer gargarejos com sal ou vinag...,Falso
...,...,...
1378,Vídeo é usado fora de contexto para relacionar...,Verdadeiro
1379,Vídeo engana ao atribuir protesto na Bélgica a...,Verdadeiro
1380,Vídeo mostra FBI confiscando máscaras de acumu...,Verdadeiro
1381,Vídeo usa informações falsas para dizer que lo...,Verdadeiro


In [ ]:
df['classification'].unique()

array(['Falso', 'Enganoso', 'Parcialmente falso', 'Distorcido',
       'Verdadeiro, mas', 'Exagerado', 'Dúbio', 'Verdadeiro'],
      dtype=object)

In [ ]:
df.groupby('classification').count()

,text
classification,
Distorcido,2
Dúbio,3
Enganoso,22
Exagerado,1
Falso,658
Parcialmente falso,14
Verdadeiro,682
"Verdadeiro, mas",1


In [ ]:
len(df)

1383

## Análise dos dados

In [ ]:
df.isna().sum()

text              0
classification    0
dtype: int64

In [ ]:
df.describe()

,text,classification
count,1383,1383
unique,1335,8
top,Vídeo é usado fora de contexto para relacionar...,Verdadeiro
freq,3,682


In [ ]:
label_true = df['classification'].unique()[-1]
df['classification'].unique()

array(['Falso', 'Enganoso', 'Parcialmente falso', 'Distorcido',
       'Verdadeiro, mas', 'Exagerado', 'Dúbio', 'Verdadeiro'],
      dtype=object)

In [ ]:
type(label_true)

str

## Pré-processamento

### Padronização da classe


*   1 -> true new
*   0 -> fake new




In [ ]:
df['classification'] = df['classification'].apply(lambda x: True if x == label_true else False)
df['classification'] = df['classification'].apply(lambda x: 1 if x else 0)
df['classification'].unique()

array([0, 1])

In [ ]:
df.groupby('classification').count()

,text
classification,
0,701
1,682


### Limpeza do texto & tokenização

In [ ]:
# Função responsável por fazer a limpeza do texto. Não devem ser retirados os acentos e nem ç's.
def clean_text(text):

  punctuations = re.escape('!"#%\'()*+,./:;<=>?@[\\]^_`{|}~')
  re_remove_brackets = re.compile(r'\{.*\}')
  re_transform_numbers = re.compile(r'\d', re.UNICODE)
  re_quotes_1 = re.compile(r"(?u)(^|\W)[‘’′`']", re.UNICODE)
  re_quotes_2 = re.compile(r"(?u)[‘’`′'](\W|$)", re.UNICODE)
  re_quotes_3 = re.compile(r'(?u)[‘’`′“”]', re.UNICODE)
  re_dots = re.compile(r'(?<!\.)\.\.(?!\.)', re.UNICODE)
  re_punctuation = re.compile(r'([,";:]){2},', re.UNICODE)
  re_hiphen = re.compile(r' -(?=[^\W\d_])', re.UNICODE)
  re_tree_dots = re.compile(u'…', re.UNICODE)
  re_punkts = re.compile(r'(\w+)([%s])([ %s])' %
                        (punctuations, punctuations), re.UNICODE)
  re_punkts_b = re.compile(r'([ %s])([%s])(\w+)' %
                          (punctuations, punctuations), re.UNICODE)
  re_punkts_c = re.compile(r'(\w+)([%s])$' % (punctuations), re.UNICODE)
  re_changehyphen = re.compile(u'–')
  re_doublequotes_1 = re.compile(r'(\"\")')
  re_doublequotes_2 = re.compile(r'(\'\')')
  re_trim = re.compile(r' +', re.UNICODE)

  text = text.lower()
  text = text.replace('\xa0', ' ')
  text = re_tree_dots.sub('...', text)
  text = re.sub('\.\.\.', '', text)
  text = re_remove_brackets.sub('', text)
  text = re_changehyphen.sub('-', text)
  text = re_transform_numbers.sub('', text)
  text = re_quotes_1.sub(r'\1"', text)
  text = re_quotes_2.sub(r'"\1', text)
  text = re_quotes_3.sub('"', text)
  text = re.sub('"', '', text)
  text = re_dots.sub('.', text)
  text = re_punctuation.sub(r'\1', text)
  text = re_hiphen.sub(' - ', text)
  text = re_punkts.sub(r'\1 \2 \3', text)
  text = re_punkts_b.sub(r'\1 \2 \3', text)
  text = re_punkts_c.sub(r'\1 \2', text)
  text = re_doublequotes_1.sub('\"', text)
  text = re_doublequotes_2.sub('\'', text)
  text = re_trim.sub(' ', text)
  text = re.sub('\W', ' ', text)
  text = re.sub('\s+', ' ', text)

  return text.strip()

In [ ]:
# Carregamento das stopwords
file = open("/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/02. Arquivos/stopwords-pt.txt", 'r')
tokens = file.readlines()
STOP_WORDS = []

for i in tokens:
    STOP_WORDS.append(i.replace('\n', ''))

# Função que padroniza e gera os tokens do texto
def text_tokens(text):
  doc = nlp(clean_text(text))
  
  return [token.text for token in doc]

# Função que remove as stopwords do texto
def remove_text_stopwords(text): 
  return [token for token in text if token not in STOP_WORDS] 

In [ ]:
tqdm.pandas(desc='Cleaning text and tokenizing')
df['text_tokens'] = df['text'].progress_apply(text_tokens)

tqdm.pandas(desc='Removing stopwords')
df['text_tokens'] = df['text_tokens'].progress_apply(remove_text_stopwords)
  
df.head()

Removing stopwords: 100%|██████████| 1383/1383 [00:00<00:00, 9608.92it/s]


,text,classification,text_tokens
0,China se beneficiou financeiramente do novo co...,0,"[china, beneficiou, financeiramente, coronavírus]"
1,Alimentos alcalinos ajudam a prevenir ou trata...,0,"[alimentos, alcalinos, ajudam, prevenir, trata..."
2,"Minas Gerais tem menos mortes que São Paulo, e...",0,"[minas, gerais, mortes, paulo, adotado, medida..."
3,"Quando os sintomas do COVID-19 aparecerem, as ...",0,"[sintomas, covid, aparecerem, tomar, azitromic..."
4,Beber água e fazer gargarejos com sal ou vinag...,0,"[beber, água, gargarejos, sal, vinagre, impede..."


## Salvando novo dataset

In [ ]:
df['text'] = df['text'].apply(lambda x: re.sub("\r", " ", x))

df.to_csv('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/02. Arquivos/01. Dados/fakenews_data_tokens.csv', index=False, header=True)